### GA Fitness Function

In [ ]:

import copy,json
from operator import itemgetter
from pprint import pprint
from visualize import plot_3d_rectangle
%matplotlib qt  
from helper import*
occupied_vol = 0
number_boxes = 0
value = 0
result = []
sorted_boxes = []
truck_dimension = [589.8,243.8, 259.1]
VOL= truck_dimension[0]*truck_dimension[1]*truck_dimension[2]
L,W,H = truck_dimension
vol_items = 0
PP = [[0,0,0]]#,[0,W,0]]
removed_PP = []
placed =[]
removed_PP = []
best_point = []
overlaped = False
lcheck,wcheck,hcheck = [False, False,False]
# Plot using matplotlib
x_ticks = 600
y_ticks = 250
z_ticks = 270
step_size =50
effective_dimention ={0:[0,1,2],1:[1,0,2],2:[0,2,1],
                        3:[1,2,0],4:[2,1,0], 5:[2,0,1]}
count = 1
support_ratio = 0.7

from box import  Item
from container import Container 
max_weight=28080

CONT = Container(
    name="ISO-20 feet Dry Container",
    LWH= (589.8,243.8,259.1),
    max_weight=28080,
)
print(CONT.string())
with open('data.json', 'r') as f:
    data = json.load(f)
# input_file = 'DataSets/processed/Updated/wtpack1.json'
# with open(input_file, 'r') as outfile:
#     data = json.load(outfile)
boxes = [] 

for i in range(len(data)):
    box,r =data[i]
    boxes.append(Item(
        partno = f"Box-{i+1}",
        name= f"C-{box[-1]}",
        weight=box[3],
        LWH = box[0:3],
        rotation = r
    ))    
boxes=sorted(boxes,key=lambda item: (item.get_volume(),item.get_dimention()[0]),reverse=True)#,item[2],item[1]
boxes=sorted(boxes,key=lambda item: (item.get_id().split("C-")[1]),reverse=False)
L,W,H = CONT.get_dimention()

for box in boxes[:]:

    l,w,h = box.get_dimention()

    for pos in CONT.PP[:]:  
        if ( L < pos[0]+l or W < pos[1]+w or H < pos[2]+h ): continue
         
        if pos == [0,0,0]:

            box.set_onBase(True)
            box.set_position(pos)
            box.is_fit = True
            CONT.fit_items.append(box)
            boxes.remove(box)
            # vol_items += box[4]
            temp_pp = []
            if pos[0]+l < L:
                temp_pp.append([pos[0]+l,pos[1],pos[2]])
            if pos[1]+w < W:
                temp_pp.append([pos[0],pos[1]+w,pos[2]])
            if pos[2]+h < H:
                temp_pp.append([pos[0],pos[1],pos[2]+h])
            CONT.PP.remove(pos)
            best_point.extend([pos])
            CONT.PP.extend(temp_pp)
            box.set_pps(temp_pp)
            CONT.PP=sorted(CONT.PP,key=lambda point: (point[0],point[2],point[1]), reverse=False)
            break   
        if ( L < pos[0]+l or W < pos[1]+w or H < pos[2]+h ): continue
        
        is_overlap = []
        support_area = []
        is_hanging = []
        temp = {}
        temp_pp = []

        for current_item in CONT.fit_items:

            if box.get_id() == 'Box-37C-2':# and pos ==[496,81,55]:#current_item.allVertices["BBL"]==[0, 192, 173]:
                pass
            box1 = current_item.get_position()+current_item.get_dimention()
            box2 = pos+box.get_dimention()
            current_item.set_allvertices(current_item.get_position()+current_item.get_dimention())
            box.set_allvertices(pos+box.get_dimention())
            box1_vertices = current_item.get_allVertices()
            box2_vertices = box.get_allVertices()
            check_overlap = intersect(box1,box2)
            is_overlap.append(check_overlap)
            if (box1_vertices["FTL"] == box2_vertices["FBL"]) and check_overlap == False:
                  if box1 not in box.under:
                       box.under.append([current_item,calculate_overlap_area(box1, box2)])
                       current_item.top.append([box,calculate_overlap_area(box1, box2)])
                       
            if (box1_vertices["BBL"] == box2_vertices["FBL"]) and check_overlap == False:
                 current_item.besideR.append(box)
                 box.besideL.append(current_item)

            if (box1_vertices["FBR"] == box2_vertices["FBL"]) and check_overlap == False:#(box1[2]+box1[5]) == box2[2]:
                current_item.front.append(box)
                box.back.append(current_item)

        if True in is_overlap:
            #removed_PP.append(pos)
            #CONT.PP.remove(pos)
            if box.besideL:
                if box.besideL[0].get_id() == 'Box-3C-1':
                    pass
            clear_neighbors(current_item,box)
            continue 
        if box.get_id() == "Box-42C-2":
            pass
        obj =None
        area = 0
        
        if box.under and pos[2] !=0:
            obj,_ = box.get_under()[0]
            area += calculate_overlap_area(obj.get_position()+obj.get_dimention(),
                                pos+box.get_dimention())
            if area < support_ratio: 
                clear_neighbors(current_item,box)
                #remove_pp(area,CONT,removed_PP,pos)
                continue
            else:
                if obj.besideR:
                    if obj.besideR[0].top:
                        top_boxes = get_top_boxes(obj.besideR[0].top)
                        
                        # cum_height = sum(b.get_dimention()[2] for b in top_boxes)
                            # print(pos+box.get_dimention() )
                            # print(f"{b.get_id()},{b.get_dimention() + b.get_position()}")
                            # pprint(b.get_allVertices())
                        #! what b?
                        d = box.get_dimention()    
                        if (box.get_dimention()[2]/top_boxes[-1].get_dimention()[2]) > support_ratio:#pos[2] == b.get_dimention()[2] + b.get_position()[2]:
                            area3 = calculate_overlap_area(top_boxes[-1].get_position() + top_boxes[-1].get_dimention(),
                                                            pos + box.get_dimention())
                            box.under.append([top_boxes[-1], area3])
                            top_boxes[-1].top.append([box, area3])
                    # if obj.besideR[0].top:
                    #         for b in obj.besideR[0].top:
                    #             if pos[2]== \
                    #                 b[0].get_dimention()[2]+b[0].get_position()[2]:
                    #                 area3 = calculate_overlap_area(b.get_position()+b.get_dimention(),
                    #                 pos+box.get_dimention())
                    #                 box.under.append([b,area3])
                    #                 b[0].top.append([box,area3]) 
                    else:
                        area3 = calculate_overlap_area(obj.besideR[0].get_position()+obj.get_dimention(),
                                        pos+box.get_dimention())
                        if area3 > 0:
                                box.under.append([obj.besideR[0],area3])
                                obj.besideR[0].top.append([box,area3]) 
        
        elif box.back and pos[2] !=0:  
            if  box.back[0].get_under():
                obj,_ = box.back[0].get_under()[0]
                area += calculate_overlap_area(obj.get_position()+obj.get_dimention(),
                                              pos+box.get_dimention())
                if area < support_ratio:
                    #remove_pp(area,CONT,removed_PP,pos) 
                    clear_neighbors(current_item,box)
                    continue
                else:
                    area3 = 0
                    box.under.append([obj,area])
                    obj.top.append([box,area])
                    if obj.besideR:
                        area3 += calculate_overlap_area(obj.besideR[0].get_position()+obj.besideR[0].get_dimention(),
                                                                    pos+box.get_dimention())                      
                        if area3 > 0:
                            box.under.append([obj.besideR[0],area3])
                            obj.besideR[0].top.append([box,area3]) 
        elif box.besideL and pos[2] !=0:

            if box.besideL[0].get_under():
                area1 = 0
                under_boxes = box.besideL[0].get_under()
                # if len(under_boxes) >= 3:
                for b in under_boxes:
                    area1 = calculate_overlap_area(b[0].get_position() + b[0].get_dimention(),
                                                    pos + box.get_dimention())
                    if area1 > 0:
                        obj, _ = b
                        break
                    # area1 += sum()
                    #                 for b in box.besideL[0].get_under())
                # else:
                    
                #     obj,_ = box.besideL[0].get_under()[0]
                #     area1 += calculate_overlap_area(obj.get_position()+obj.get_dimention(),
                #                                     pos+box.get_dimention())
                area2 = 0
                if area1 < .5 and obj!=None:
                    if obj.besideR:
                        area2 += calculate_overlap_area(obj.besideR[0].get_position()+obj.besideR[0].get_dimention(),
                                                    pos+box.get_dimention())

                area =area1+area2
                
                if area1!=0 and area1 > 0.7:
                    area3 = 0
                    box.under.append([obj,area1])
                    obj.top.append([box,area1])
                    if obj.besideR and calculate_overlap_area(obj.besideR[0].get_position()+obj.besideR[0].get_dimention(),
                                                    pos+box.get_dimention())>0:
                        area3 = calculate_overlap_area(obj.besideR[0].get_position()+obj.besideR[0].get_dimention(),
                                                    pos+box.get_dimention())
                        box.under.append([obj.besideR[0],area3])
                        obj.besideR[0].top.append([box,area3])
                if area2 !=0 and area2 > 0.7:
                    box.under.append([obj.besideR[0],area2])
                    obj.besideR[0].top.append([box,area2])

            if (area) < support_ratio:
                clear_neighbors(current_item,box)
                #remove_pp(area,CONT,removed_PP,pos)
                continue            
        else:
            pass
        # else:
        #     obj = box.back[0]    
        #     area += calculate_overlap_area(obj.get_position()+obj.get_dimention(),
        #                                     pos+box.get_dimention())                                                                              
        #     if area < support_ratio:
        #         #remove_pp(area,CONT,removed_PP,pos) 
        #         clear_neighbors(current_item,box)
        #         continue        
        # elif box.besideL and pos[2] !=0:
        #     if box.besideL[0].get_under():
        #         obj,_ = box.besideL[0].get_under()[0]
        #         area += calculate_overlap_area(obj.get_position()+obj.get_dimention(),
        #                                     pos+box.get_dimention())
        #         box.under.append([obj,area])
        #         if obj.besideR and area < .4: 
        #             area += calculate_overlap_area(obj.besideR[0].get_position()+obj.get_dimention(),
        #                                     pos+box.get_dimention())
        #             area1 = calculate_overlap_area(obj.besideR[0].get_position()+obj.get_dimention(),
        #                                     pos+box.get_dimention())
        #             box.under.append([obj.besideR,area1])
            # else:
            #     obj,_ = box.besideL[0].get_under()[0]
            #     area += calculate_overlap_area(obj.get_position()+obj.get_dimention(),
            #                                   pos+box.get_dimention())

        # if (area) < support_ratio:
        #     clear_neighbors(current_item,box)
        #     #remove_pp(area,CONT,removed_PP,pos)
        #     continue                            
    
        if pos[2] == 0:box.set_onBase(True)
        box.set_position(pos)
        box.is_fit = True
        CONT.fit_items.append(box)
        #boxes.remove(box)
        if pos[0]+l < L and [pos[0]+l,pos[1],pos[2]] not in CONT.PP:
           temp_pp.append( [pos[0]+l,pos[1],pos[2]])
        if pos[1]+w < W and [pos[0],pos[1]+w,pos[2]] not in CONT.PP:
            temp_pp.append([pos[0],pos[1]+w,pos[2]])
        if pos[2]+h < H and [pos[0],pos[1],pos[2]+h] not in CONT.PP:
            temp_pp.append([pos[0],pos[1],pos[2]+h])

        CONT.PP.remove(pos)
        best_point.extend([pos])
        CONT.PP.extend(temp_pp)
        box.set_pps(temp_pp)
        CONT.PP=sorted(CONT.PP,key=lambda point: (point[0],point[2],point[1]), reverse=False)

        # result.append(f"{box.get_id()}-{count}")#print(f"{box.get_id()}-{count}")
        # count +=1
        break 
    
#! ////////////////// Debuging//////////////////    
# for item in boxes:
#     if item.is_fit == False:
#         print(item.get_id())
#         print(item.get_LWH_R())
#         print(item.get_dimention())
#         print(item.get_id())
    bx = [box.get_plot_data() for box in CONT.fit_items]
    plot_3d_rectangle(bx, x_ticks, y_ticks, z_ticks, step_size,0)
#print(CONT.fit_items)
# with open('best_ppV3.json', 'w') as json_file:
#     json_file.write((json.dumps(result)))
# with open('best_ppV2.json', 'w') as json_file:
#     json_file.write((json.dumps(CONT.fit_items)))
    


### Check Neighbours Manually

In [24]:
plot_3d_rectangle([[92,0,92,43,110,25,1,"Box-24"],[92,110,92,25,43,110,1,"Box-19"],
                   [168,81,30,30,108,76,2,"Box-42"],[92,81,30,76,108,30,1,"Box-11"],
                   [92,81,0,108,76,30,1,"Box-12"],[92,157,0,108,76,30,1,"Box-14"],
                   [168,81,106,30,108,76,3,"Box-64"]]
                  ,x_ticks, y_ticks, z_ticks, step_size,0)

### Plot solutions from JSON

In [11]:
x_ticks = 600
y_ticks = 250
z_ticks = 270
step_size =50
with open('truck_and_packages.json', 'r') as json_file:
    data = json.load(json_file)
#m,k,v = list(data.keys())[1:len(list(data.keys()))]
v = list(data.keys())[1:]
bx = [box["plot_data"] for box in data.get(v[3]).get("item_passport")]

#*to plot boxes step by step
# for i in range(len(bx)):
    # plot_3d_rectangle(bx[0:i+1], x_ticks, y_ticks, z_ticks, step_size,0)
#

plot_3d_rectangle(bx, x_ticks, y_ticks, z_ticks, step_size,0)

#### For Visa


In [3]:
result = {
    "cargo_metadata": {
            "container_dimension": [CONT.get_dimention()],
            "container_weight": 28080,
            "container_volume": CONT.get_volume(),
            "total_items": len(boxes),
            "packed_items": len(CONT.fit_items),
            "unpacked_items": len([i.is_fit == False for i in boxes]),
            "fitness": ["Vol_utilization(float)", "packed boxes", "Average_Value"],
            "consignment_origin": ["City ABC"],
            "consignment_destinations": ["ABC", "ABC", "ABC"]
        },
    "00": {
        "item_passport": [
            {
                "id": item.get_id(),
                "customer_belonging": item.name,
                "volume": item.get_volume(),
                "weight": item.weight,
                "material": "",
                "rotation_type": item.rotation_type,
                "dimention_before_rotation": item.get_LWH_R()[0:3],
                "dimention_after_rotation": item.get_dimention()[0:3],
                "plot_data": item.get_plot_data(),
                "on_base":item.onBase,"stackable":"",
                "neighboring_items": {
                    "left": [neighbor.get_id() for neighbor in item.besideL if neighbor],#.besideL],
                    "right": [neighbor.get_id() for neighbor in item.besideR if neighbor],#.besideR],
                    "top": [[neighbor[0].get_id(),neighbor[1]] for neighbor in item.top if neighbor],
                    "under": [[neighbor[0].get_id(),neighbor[1]] for neighbor in item.under if neighbor],
                    "back": [neighbor.get_id() for neighbor in item.back if neighbor],#.back],
                    "front": [neighbor.get_id() for neighbor in item.front if neighbor],#.front]
                }
            } for item in CONT.fit_items
        ]
    }
}
#result
#[neighbor[0].get_id(),neighbor[0][1]]
#[item.under[0][0].get_id(),item.under[0][1]]
with open('truck_and_packagesV2.json', 'w') as json_file:
    json_file.write((json.dumps(result, indent=4)))

### Check Trouble making neighbours

In [ ]:
for item in CONT.fit_items:
    if item.get_id() == "Box-11C-1":
        pass
    l=[neighbor.get_id() for neighbor in item.besideL if neighbor]#.besideL]
    r = [neighbor.get_id() for neighbor in item.besideR if neighbor]#.besideR]
    t=[[neighbor[0].get_id(),neighbor[1]] for neighbor in item.top if neighbor]
    u=[[neighbor[0].get_id(),neighbor[1]] for neighbor in item.under if neighbor]
    b=[neighbor.get_id() for neighbor in item.back if neighbor]#.back]
    f=[neighbor.get_id() for neighbor in item.front if neighbor]#.front]
    print(item.get_id(),l,r,t,u,b,f)            

### Get data from item_passport

In [ ]:
from pprint import pprint
import json
file_name = "truck_and_packagesV2"

with open(f'{file_name}.json', 'r') as json_file:
    data = json.load(json_file)
item_passport= data.get("00").get("item_passport")
selected_dict = next((d for d in item_passport if d["id"] == "Box-18C-1"), None)
pprint(selected_dict)
selected_dict = next((d for d in item_passport if d["id"] == "Box-15C-1"), None)
pprint(selected_dict)
selected_dict = next((d for d in item_passport if d["id"] == "Box-45C-2"), None)
pprint(selected_dict)

#### Only for plotting PP on rectangles


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt 
x_ticks = 600
y_ticks = 250
z_ticks = 250
step_size =50
decriment=30
def plot_rectangles_with_vertices(rectangles):
    plt.interactive(True)
    fig = fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111, projection='3d')

    for rect in rectangles:
        x, y, z, l, w, h,_ = rect

        # Plot the rectangle
        ax.bar3d(x, y, z, l, w, h,alpha=.2, edgecolor=(0, 0, 0, 1), lw=3)

        # Generate and plot vertices
        vertices = [
            (x + l, y, z),  # (xi+li, yi, zi)
            (x, y + w, z),  # (xi, yi+wi, zi)
            (x, y, z + h)   # (xi, yi, zi+hi)
        ]
        for vertex in vertices:
            ax.scatter(*vertex, color='red', s=50)  # Plot a small circle at each vertex
    ax.set_xticks(np.arange(0, x_ticks + step_size-decriment, step_size-decriment))
    ax.set_yticks(np.arange(0, y_ticks + step_size-40, step_size-40))
    ax.set_zticks(np.arange(0, z_ticks + step_size-40, step_size-40))
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Rectangles with Vertices')

    plt.show()

# Example list of rectangles [x, y, z, length, width, height]
rectangles = [[0,0,0,55,92,81,1][0,92,0,81,92,55,1],[0,92,55,81,55,92,1],
              [0,147,55,92,81,55,1],[0,0,81,55,81,92,1],[0,147,110,92,81,55,1],
              [0,92,147,92,55,81,1],[0,147,165,92,81,55,1],[0,0,173,92,81,55,1],
              [55,0,0,92,55,81,1],[81,92,0,76,108,30,1],[55,0,81,108,76,30,1],
              [55,55,0,108,30,76,1]]

# Plot rectangles with their vertices
plot_rectangles_with_vertices(rectangles)
